In [0]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx
from scipy.stats import poisson, norm, uniform
import csv
import numpy as np



#Function to add cities as nodes
def add_city(graph,name,area,population,cases,neighbours):


  #neighbours is a list of tuples. Each tuple contains the name of the city,
  #the area, the population, its neighbours and the strength between each of its
  #neighbours (weight). See the second cell for the format of the data

  #subtract the cases from the population to get the uninfected population
  population -= cases
  population_density = population/area

  #Add node to graph
  graph.add_node(name,population=population,
                 population_density=population_density,
                 cases=cases)
  
  #Add edges between the neighbours
  graph.add_weighted_edges_from(neighbours)

#Function that simulates a day
def simulate_day(graph,lambda_,recovery):

  #Sample 1 million numbers first instead of sampling each time the for loop
  #runs because it is much faster
  poisson_samples = poisson.rvs(lambda_,size=1000000)
  uniform_samples = uniform.rvs(size=1000000)
  #Loop through nodes
  for node in graph.nodes:

    #Loop through each case and sample from a poisson distribution to see how
    #many people the case infected

    for people in range(graph.nodes[node]['cases']):
      spread = np.random.choice(poisson_samples)
      graph.nodes[node]['cases'] += spread
      graph.nodes[node]['population'] -= spread


    #Multiplier effect due to density of city
    spread = int(round(graph.nodes[node]['population_density'] * abs(norm.rvs(0,0.001) * graph.nodes[node]['cases'])))
    graph.nodes[node]['cases'] += spread
    graph.nodes[node]['population'] -= spread  

    #Small chance of recovery in people
    recovered = 0
    recovery_rate = round(abs(norm.rvs(recovery,0.05)))
    for people in range(graph.nodes[node]['cases']):
      if np.random.choice(uniform_samples) < recovery_rate:
        recovered +=1
    graph.nodes[node]['cases'] -= recovered
    graph.nodes[node]['population'] += recovered


    #Loop through each edge of each node to simulate flow of people
    for edges in graph.adj[node]:
      #get the weight
      weight =  list(graph[node][edges]['weight'].values())[0]

      #get expected flow of traffic from the city
      expected_flow = round((weight + abs(norm.rvs(0,0.01))) *  (graph.nodes[node]['cases']+graph.nodes[node]['population']))

      #check how many infected people travel
      samples = uniform.rvs(size=int(expected_flow))
      corona_flow = 0
      for i in samples:
        if i < graph.nodes[node]['cases']/graph.nodes[node]['population']:
          corona_flow += 1
      #print(edges)
      #print(node)
      #print(graph.nodes[node])
      #print('-----')
      graph.nodes[node]['cases'] -= corona_flow
      graph.nodes[edges]['cases'] += corona_flow

In [0]:
#Cell for adding all districts in Pakistan

def make_Pakistan():
#Make Pakistan graph
  Pakistan = nx.Graph()

  #Add cities/districts

  #data was taken from https://en.wikipedia.org/wiki/Districts_of_Sindh,_Pakistan

  """Sindh"""

  add_city(Pakistan, 'Badin',6470,1804516,0, [("Badin",'Thatta',{'weight':abs(norm.rvs(0.001,0.0005))}),("Badin",'Tando Muhammad Khan',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Badin",'Hyderabad',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Badin",'Tando Allahyar',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Badin",'Mirpur Khas',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Badin",'Tharparker',{'weight':abs(norm.rvs(0.001,0.0005))})] )

  add_city(Pakistan, 'Dadu',8034,1550266,1, [("Dadu",'Qambar Shahdadkot',{'weight':abs(norm.rvs(0.001,0.0005))}),("Dadu",'Larkana',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                            ("Dadu",'Larkana',{'weight':abs(norm.rvs(0.001,0.0005))}),("Dadu",'Naushahro Firoz',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                            ("Dadu",'Jamshoro',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Dadu",'Khuzdar',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                            ("Dadu",'Lasbela',{'weight':abs(norm.rvs(0.001,0.0005))})])

  add_city(Pakistan, 'Ghotki',6506,1647239,0, [("Ghotki",'Kashmore',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Ghotki",'Sukkur',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Ghotki",'Rahim Yar Khan',{'weight':abs(norm.rvs(0.001,0.0005))})] )

  add_city(Pakistan, 'Hyderabad',1022,2201079,2, [("Hyderabad",'Jamshoro',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Hyderabad",'Matiari',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Hyderabad",'Sanghar',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Hyderabad",'Tando Allahyar',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Hyderabad",'Badin',{'weight':abs(norm.rvs(0.001,0.0005))}),("Hyderabad",'Tando Muhammad Khan',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Hyderabad",'Thatta',{'weight':abs(norm.rvs(0.001,0.0005))})] )

  add_city(Pakistan, 'Jacobabad',2771,1006297,0, [("Jacobabad",'Kashmore',{'weight':abs(norm.rvs(0.001,0.0005))}),("Jacobabad",'Shikarpur',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Jacobabad",'Larkana',{'weight':abs(norm.rvs(0.001,0.0005))}),("Jacobabad",'Qambar Shahdadkot',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Jacobabad",'Jafarabad',{'weight':abs(norm.rvs(0.001,0.0005))})] )

  add_city(Pakistan, 'Jamshoro',11250,993142,0, [ ("Jamshoro",'Dadu',{'weight':abs(norm.rvs(0.001,0.0005))}),  ("Jamshoro",'Naushahro Firoz',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Jamshoro",'Nawabshah',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Jamshoro",'Matiari',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Jamshoro",'Hyderabad',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Jamshoro",'Thatta',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Jamshoro",'Karachi',{'weight':abs(norm.rvs(0.001,0.0005))}),("Jamshoro",'Lasbela',{'weight':abs(norm.rvs(0.001,0.0005))})] )


  add_city(Pakistan, 'Karachi',3672,17144157,132, [ ("Karachi",'Jamshoro',{'weight':abs(norm.rvs(0.0007,0.0003))}),  ("Karachi",'Thatta',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                  ("Karachi",'Lasbela',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Kashmore',2551,1089169,0, [ ("Kashmore",'Ghotki',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Kashmore",'Sukkur',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Kashmore",'Shikarpur',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Kashmore",'Jacobabad',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Kashmore",'Dera Ghazi Khan',{'weight':abs(norm.rvs(0.0007,0.0003))}),  ("Kashmore",'Rajanpur',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Kashmore",'Dera Bugti',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Khairpur',15925,2405523,0, [("Khairpur",'Larkana',{'weight':abs(norm.rvs(0.001,0.0005))}),("Khairpur",'Sukkur',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Khairpur",'Shikarpur',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Khairpur",'Nawabshah',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Khairpur",'Sanghar',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Khairpur",'Naushahro Firoz',{'weight':abs(norm.rvs(0.001,0.0005))})] )

  add_city(Pakistan, 'Larkana',1906,1524391,0, [("Larkana",'Shikarpur',{'weight':abs(norm.rvs(0.001,0.0005))}),("Larkana",'Jacobabad',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Larkana",'Qambar Shahdadkot',{'weight':abs(norm.rvs(0.001,0.0005))}),("Larkana",'Dadu',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Larkana",'Naushahro Firoz',{'weight':abs(norm.rvs(0.001,0.0005))}),("Larkana",'Khairpur',{'weight':abs(norm.rvs(0.001,0.0005))})] )

  add_city(Pakistan, 'Matiari',1459,769349,0, [("Matiari",'Nawabshah',{'weight':abs(norm.rvs(0.001,0.0005))}),("Matiari",'Sanghar',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                            ("Matiari",'Hyderabad',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Matiari",'Jamshoro',{'weight':abs(norm.rvs(0.001,0.0005))})] )

  add_city(Pakistan, 'Mirpur Khas',3319,1505876,0, [("Mirpur Khas",'Sanghar',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Mirpur Khas",'Umerkot',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                  ("Mirpur Khas",'Tharparker',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Mirpur Khas",'Tando Allahyar',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                  ("Mirpur Khas",'Badin',{'weight':abs(norm.rvs(0.001,0.0005))})] )

  add_city(Pakistan, 'Naushahro Firoz',2027,1612373,0, [("Naushahro Firoz",'Dadu',{'weight':abs(norm.rvs(0.001,0.0005))}),("Naushahro Firoz",'Larkana',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                      ("Naushahro Firoz",'Khairpur',{'weight':abs(norm.rvs(0.001,0.0005))}),("Naushahro Firoz",'Nawabshah',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                      ("Naushahro Firoz",'Jamshoro',{'weight':abs(norm.rvs(0.001,0.0005))})])

  add_city(Pakistan, 'Nawabshah',4618,2012847,0, [("Nawabshah",'Khairpur',{'weight':abs(norm.rvs(0.001,0.0005))}),("Nawabshah",'Sanghar',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Nawabshah",'Naushahro Firoz',{'weight':abs(norm.rvs(0.001,0.0005))}),("Nawabshah",'Matiari',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Nawabshah",'Jamshoro',{'weight':abs(norm.rvs(0.001,0.0005))})] )

  add_city(Pakistan, 'Qambar Shahdadkot',5599,1341042,0, [("Qambar Shahdadkot",'Larkana',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Qambar Shahdadkot",'Dadu',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                        ("Qambar Shahdadkot",'Jacobabad',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Qambar Shahdadkot",'Jafarabad',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                        ("Qambar Shahdadkot",'Jhal Magsi',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Qambar Shahdadkot",'Khuzdar',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Sanghar',10259,2057057	,0, [("Sanghar",'Khairpur',{'weight':abs(norm.rvs(0.001,0.0005))}),("Sanghar",'Matiari',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Sanghar",'Nawabshah',{'weight':abs(norm.rvs(0.001,0.0005))}),("Sanghar",'Hyderabad',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Sanghar",'Mirpur Khas',{'weight':abs(norm.rvs(0.001,0.0005))}),("Sanghar",'Tando Muhammad Khan',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Sanghar",'Umerkot',{'weight':abs(norm.rvs(0.001,0.0005))})] )

  add_city(Pakistan, 'Shikarpur',2577,1231481,0, [("Shikarpur",'Jacobabad',{'weight':abs(norm.rvs(0.001,0.0005))}),("Shikarpur",'Kashmore',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Shikarpur",'Ghotki',{'weight':abs(norm.rvs(0.001,0.0005))}),("Shikarpur",'Larkana',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Shikarpur",'Sukkur',{'weight':abs(norm.rvs(0.001,0.0005))}),("Shikarpur",'Khairpur',{'weight':abs(norm.rvs(0.001,0.0005))})] )

  add_city(Pakistan, 'Sukkur',5216,1487903,268, [("Sukkur",'Ghotki',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Sukkur",'Kashmore',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                            ("Sukkur",'Shikarpur',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Sukkur",'Khairpur',{'weight':abs(norm.rvs(0.0007,0.00035))})] )

  add_city(Pakistan, 'Tando Allahyar',1573,836887,0, [("Tando Allahyar",'Sanghar',{'weight':abs(norm.rvs(0.001,0.0005))}),("Tando Allahyar",'Mirpur Khas',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                    ("Tando Allahyar",'Badin',{'weight':abs(norm.rvs(0.001,0.0005))}),("Tando Allahyar",'Hyderabad',{'weight':abs(norm.rvs(0.001,0.0005))})] )

  add_city(Pakistan, 'Tando Muhammad Khan',1814,677228,0, [("Tando Muhammad Khan",'Hyderabad',{'weight':abs(norm.rvs(0.001,0.0005))}),("Tando Muhammad Khan",'Thatta',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                          ("Tando Muhammad Khan",'Badin',{'weight':abs(norm.rvs(0.001,0.0005))})] )

  add_city(Pakistan, 'Tharparker',19808,1649661,0, [("Tharparker",'Umerkot',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Tharparker",'Badin',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                  ("Tharparker",'Mirpur Khas',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Thatta',16404,1761784,0, [("Thatta",'Hyderabad',{'weight':abs(norm.rvs(0.001,0.0005))}),("Thatta",'Jamshoro',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Thatta",'Karachi',{'weight':abs(norm.rvs(0.001,0.0005))}),("Thatta",'Tando Muhammad Khan',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Thatta",'Badin',{'weight':abs(norm.rvs(0.001,0.0005))})] )

  add_city(Pakistan, 'Umerkot',5503,1073146,0, [("Umerkot",'Sanghar',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Umerkot",'Mirpur Khas',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Umerkot",'Tharparker',{'weight':abs(norm.rvs(0.0007,0.0003))})] )
  
  """ Islamabad """
  add_city(Pakistan, 'Islamabad',906,2006572,11, [("Islamabad",'Rawalpindi',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Islamabad",'Haripur',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  """Punjab"""
  add_city(Pakistan, 'Attock',6858,1883556,0, [("Attock",'Chakwal',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Attock",'Mianwali',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Attock",'Rawalpindi',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Attock",'Kohat',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Attock",'Mardan',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Attock",'Shangla',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Attock",'Haripur',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Bahawalnagar',8878,2981919,0, [("Bahawalnagar",'Okara',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Bahawalnagar",'Pakpattan',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                    ("Bahawalnagar",'Vehari',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Bahawalnagar",'Bahawalpur',{'weight':abs(norm.rvs(0.0007,0.0003))})])

  add_city(Pakistan, 'Bahawalpur',24830,3668106,0, [("Bahawalpur",'Rahim Yar Khan',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Bahawalpur",'Muzaffargarh',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                    ("Bahawalpur",'Multan',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Bahawalpur",'Lodhran',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                    ("Bahawalpur",'Vehari',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Bahawalpur",'Bahawalnagar',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Bhakkar',8153,1650518,0, [("Bhakkar",'Layyah',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Bhakkar",'Jhang',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Bhakkar",'Khushab',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Bhakkar",'Mianwali',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Bhakkar",'Dera Ismail Khan',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Chakwal',6524,1495982,0, [("Chakwal",'Attock',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Chakwal",'Rawalpindi',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Chakwal",'Jhelum',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Chakwal",'Khushab',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Chakwal",'Mianwali',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Chiniot',2643,1369740,0, [("Chiniot",'Sargodha',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Chiniot",'Hafizabad',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Chiniot",'Faisalabad',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Chiniot",'Jhang',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Dera Ghazi Khan',11922,2872201,198, [("Dera Ghazi Khan",'Layyah',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Dera Ghazi Khan",'Muzaffargarh',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                        ("Dera Ghazi Khan",'Rajanpur',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Dera Ghazi Khan",'Kashmore',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                        ("Dera Ghazi Khan",'Dera Ismail Khan',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Dera Ghazi Khan",'FR Dera Ismail Khan',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                        ("Dera Ghazi Khan",'Musakhel',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Dera Ghazi Khan",'Barkhan',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                        ("Dera Ghazi Khan",'Dera Bugti',{'weight':abs(norm.rvs(0.0007,0.0003))})] )


  add_city(Pakistan, 'Faisalabad',5856,7873910,0, [("Faisalabad",'Nankana Sahib',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Faisalabad",'Hafizabad',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                  ("Faisalabad",'Okara',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Faisalabad",'Sahiwal',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                  ("Faisalabad",'Toba Tek Singh',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Faisalabad",'Jhang',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                  ("Faisalabad",'Chiniot',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Gujranwala',3622,5014196,7, [("Gujranwala",'Gujrat',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Gujranwala",'Sialkot',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                  ("Gujranwala",'Sheikhupura',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Gujranwala",'Hafizabad',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                  ("Gujranwala",'Mandi Bahauddin',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Gujrat',3192,2756110,12, [("Gujrat",'Jhelum',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Gujrat",'Mandi Bahauddin',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Gujrat",'Gujranwala',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Gujrat",'Sialkot',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Gujrat",'Kashmir',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Hafizabad',2367,1156957,0, [("Hafizabad",'Mandi Bahauddin',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Hafizabad",'Gujranwala',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                  ("Hafizabad",'Sheikhupura',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Hafizabad",'Nankana Sahib',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                  ("Hafizabad",'Faisalabad',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Hafizabad",'Chiniot',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                  ("Hafizabad",'Sargodha',{'weight':abs(norm.rvs(0.0007,0.0003))})] )


  add_city(Pakistan, 'Jhang',8809,2743416,0, [("Jhang",'Chiniot',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Jhang",'Sargodha',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Jhang",'Toba Tek Singh',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Jhang",'Faisalabad',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Jhang",'Khanewal',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Jhang",'Layyah',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Jhang",'Bhakkar',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Jhang",'Khushab',{'weight':abs(norm.rvs(0.0007,0.0003))})])

  add_city(Pakistan, 'Jhelum',3587,1222650,3, [("Jhelum",'Gujrat',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Jhelum",'Mandi Bahauddin',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Jhelum",'Sargodha',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Jhelum",'Khushab',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Jhelum",'Chakwal',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Jhelum",'Rawalpindi',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Jhelum",'Kashmir',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Kasur',3995,3454996,0, [("Kasur",'Lahore',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Kasur",'Nankana Sahib',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Kasur",'Okara',{'weight':abs(norm.rvs(0.0007,0.0003))})] )


  add_city(Pakistan, 'Khanewal',4349,2921986,0, [("Khanewal",'Jhang',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Khanewal",'Toba Tek Singh',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Khanewal",'Sahiwal',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Khanewal",'Vehari',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Khanewal",'Lodhran',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Khanewal",'Multan',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Khanewal",'Layyah',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Khanewal",'Muzaffargarh',{'weight':abs(norm.rvs(0.0007,0.0003))})] )


  add_city(Pakistan, 'Khushab',6511,1281299,0, [("Khushab",'Mianwali',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Khushab",'Chakwal',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Khushab",'Jhelum',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Khushab",'Sargodha',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Khushab",'Jhang',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Khushab",'Bhakkar',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Lahore',1772,11126285,59, [("Lahore",'Sheikhupura',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Lahore",'Nankana Sahib',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Lahore",'Kasur',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Layyah',6291,1824230,0, [("Layyah",'Bhakkar',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Layyah",'Jhang',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Layyah",'Khanewal',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Layyah",'Dera Ghazi Khan',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Layyah",'Muzaffargarh',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Layyah",'Dera Ismail Khan',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Lodhran',2778,1700620,0, [("Lodhran",'Khanewal',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Lodhran",'Vehari',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Lodhran",'Multan',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Lodhran",'Bahawalpur',{'weight':abs(norm.rvs(0.0007,0.0003))})] )


  add_city(Pakistan, 'Mandi Bahauddin',2673,1593292,0, [("Mandi Bahauddin",'Jhelum',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Mandi Bahauddin",'Gujrat',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                        ("Mandi Bahauddin",'Gujranwala',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Mandi Bahauddin",'Hafizabad',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                        ("Mandi Bahauddin",'Sargodha',{'weight':abs(norm.rvs(0.0007,0.0003))})] )


  add_city(Pakistan, 'Mianwali',5840,1546094,0, [("Mianwali",'Attock',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Mianwali",'Chakwal',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Mianwali",'Khushab',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Mianwali",'Bhakkar',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Mianwali",'Dera Ismail Khan',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Mianwali",'Karak',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Mianwali",'Upper Kohistan',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Mianwali",'Kohat',{'weight':abs(norm.rvs(0.0007,0.0003))})] )


  add_city(Pakistan, 'Multan',3720,4745109,0, [("Multan",'Khanewal',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Multan",'Lodhran',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Multan",'Bahawalpur',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Multan",'Muzaffargarh',{'weight':abs(norm.rvs(0.0007,0.0003))}) ] )

  add_city(Pakistan, 'Muzaffargarh',8249,4322009,0, [("Muzaffargarh",'Layyah',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Muzaffargarh",'Khanewal',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                    ("Muzaffargarh",'Multan',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Muzaffargarh",'Bahawalpur',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                    ("Muzaffargarh",'Rahim Yar Khan',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Muzaffargarh",'Rajanpur',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                    ("Muzaffargarh",'Dera Ghazi Khan',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Narowal',2337,1709757,0, [("Narowal",'Sialkot',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Narowal",'Sheikhupura',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Nankana Sahib',2960,1356374,0, [("Nankana Sahib",'Sheikhupura',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Nankana Sahib",'Hafizabad',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                      ("Nankana Sahib",'Lahore',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Nankana Sahib",'Kasur',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                      ("Nankana Sahib",'Okara',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Nankana Sahib",'Faisalabad',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Okara',4377,3039139,0, [("Okara",'Kasur',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Okara",'Nankana Sahib',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Okara",'Sahiwal',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Okara",'Faisalabad',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Okara",'Pakpattan',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Okara",'Bahawalnagar',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Pakpattan',2724,1823687,0, [("Pakpattan",'Okara',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Pakpattan",'Sahiwal',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                  ("Pakpattan",'Vehari',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Pakpattan",'Bahawalnagar',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Rahim Yar Khan',11880,4814006,0, [("Rahim Yar Khan",'Bahawalpur',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Rahim Yar Khan",'Rajanpur',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                        ("Rahim Yar Khan",'Muzaffargarh',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Rahim Yar Khan",'Ghotki',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                        ("Rahim Yar Khan",'Kashmore',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Rajanpur',12319,1995958,0, [("Rajanpur",'Rahim Yar Khan',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Rajanpur",'Muzaffargarh',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                  ("Rajanpur",'Dera Ghazi Khan',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Rajanpur",'Kashmore',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Rawalpindi',5286,10007821,2, [("Rawalpindi",'Attock',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Rawalpindi",'Chakwal',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                    ("Rawalpindi",'Jhelum',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Rawalpindi",'Islamabad',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                    ("Rawalpindi",'Kashmir',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Sahiwal',3201,2517560,0, [("Sahiwal",'Faisalabad',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Sahiwal",'Okara',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Sahiwal",'Pakpattan',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Sahiwal",'Vehari',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Sahiwal",'Khanewal',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Sahiwal",'Toba Tek Singh',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Sargodha',5854,3703588,1, [("Sargodha",'Jhelum',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Sargodha",'Mandi Bahauddin',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Sargodha",'Hafizabad',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Sargodha",'Chiniot',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Sargodha",'Jhang',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Sargodha",'Khushab',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Sheikhupura',5960,3460426,0, [("Sheikhupura",'Sialkot',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Sheikhupura",'Narowal',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                    ("Sheikhupura",'Gujranwala',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Sheikhupura",'Hafizabad',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                    ("Sheikhupura",'Nankana Sahib',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Sheikhupura",'Lahore',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Sialkot',3016,3893672,0, [("Sialkot",'Gujranwala',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Sialkot",'Gujrat',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Sialkot",'Sheikhupura',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Sialkot",'Narowal',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Toba Tek Singh',3252,2190015,0, [("Toba Tek Singh",'Faisalabad',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Toba Tek Singh",'Jhang',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                      ("Toba Tek Singh",'Sahiwal',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Toba Tek Singh",'Khanewal',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Vehari',4365,2897446,0, [("Vehari",'Sahiwal',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Vehari",'Pakpattan',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Vehari",'Bahawalnagar',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Vehari",'Khanewal',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Vehari",'Lodhran',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Vehari",'Bahawalpur',{'weight':abs(norm.rvs(0.0007,0.0003))})] )
  
  """FATA"""
  add_city(Pakistan, 'Bajaur',120,1093684,0, [("Bajaur",'Lower Kohistan',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Bajaur",'Lower Dir',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Bajaur",'Mohmand',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Khyber',2576,986973,0, [("Khyber",'Mohmand',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Khyber",'Kurram',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Khyber",'Orakzai',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Khyber",'Nowshera',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Kurram',3380,619553,2, [("Kurram",'North Waziristan',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Kurram",'Khyber',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Kurram",'Orakzai',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Kurram",'Hangu',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Mohmand',2296,466984,0, [("Mohmand",'Khyber',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Mohmand",'Bajaur',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Mohmand",'Nowshera',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Mohmand",'Lower Dir',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Mohmand",'Charsadda',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'North Waziristan',4707,543254,0, [("North Waziristan",'FR Bannu',{'weight':abs(norm.rvs(0.0007,0.0003))}),("North Waziristan",'Kurram',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                        ("North Waziristan",'South Waziristan',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Orakzai',1538,254356,0, [("Orakzai",'Khyber',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Orakzai",'Kurram',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Orakzai",'FR Kohat',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Orakzai",'Kohat',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Orakzai",'Hangu',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'South Waziristan',6620,679185,2, [("South Waziristan",'North Waziristan',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("South Waziristan",'FR Lakki Marwat',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                        ("South Waziristan",'FR Tank',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("South Waziristan",'FR Dera Ismail Khan',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                        ("South Waziristan",'Sherani',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'FR Bannu',745,43114,0, [("FR Bannu",'North Waziristan',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("FR Bannu",'FR Lakki Marwat',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("FR Bannu",'Hangu',{'weight':abs(norm.rvs(0.0007,0.0003))}),("FR Bannu",'Karak',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("FR Bannu",'Bannu',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'FR Dera Ismail Khan',2008,68556,4, [("FR Dera Ismail Khan",'South Waziristan',{'weight':abs(norm.rvs(0.0007,0.0003))}),("FR Dera Ismail Khan",'Musakhel',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                          ("FR Dera Ismail Khan",'Sherani',{'weight':abs(norm.rvs(0.0007,0.0003))}),("FR Dera Ismail Khan",'Dera Ismail Khan',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                          ("FR Dera Ismail Khan",'Dera Ghazi Khan',{'weight':abs(norm.rvs(0.0007,0.0003))}),("FR Dera Ismail Khan",'Swat',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'FR Kohat',446,118578,0, [("FR Kohat",'FR Peshawer',{'weight':abs(norm.rvs(0.0007,0.0003))}),("FR Kohat",'Orakzai',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("FR Kohat",'Kohat',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'FR Lakki Marwat',132,26359,0, [("FR Lakki Marwat",'FR Bannu',{'weight':abs(norm.rvs(0.0007,0.0003))}),("FR Lakki Marwat",'South Waziristan',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                    ("FR Lakki Marwat",'FR Tank',{'weight':abs(norm.rvs(0.0007,0.0003))}),  ("FR Lakki Marwat",'Bannu',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                      ("FR Lakki Marwat",'Upper Kohistan',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'FR Peshawer',261,64691,2, [("FR Peshawer",'FR Kohat',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("FR Peshawer",'Kohat',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("FR Peshawer",'Mardan',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("FR Peshawer",'Nowshera',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'FR Tank',1221,36389,0, [("FR Tank",'FR Lakki Marwat',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("FR Tank",'South Waziristan',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("FR Tank",'Upper Kohistan',{'weight':abs(norm.rvs(0.0007,0.0003))}),("FR Tank",'Swat',{'weight':abs(norm.rvs(0.0007,0.0003))})] )
  
  """Kashmir (single entity for this simulation)"""

  add_city(Pakistan, 'Kashmir',13297,4045366,1, [("Kashmir",'Gujrat',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Kashmir",'Jhelum',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                               ("Kashmir",'Rawalpindi',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Kashmir",'Malakand',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                               ("Kashmir",'Abbottabad',{'weight':abs(norm.rvs(0.0007,0.0003))}),("Kashmir",'Skardu',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                               ("Kashmir",'Astore',{'weight':abs(norm.rvs(0.0007,0.0003))}), ("Kashmir",'Diamer',{'weight':abs(norm.rvs(0.0007,0.0003))})] )
  
  """Gilgit-Baltistan"""
  #Data was only available from 1998 for some regions. The 1998 data was multiplied by a factor of ~1.5 to account for the present population

  add_city(Pakistan, 'Ghanche',6400,150000,0, [("Ghanche",'Skardu',{'weight':abs(norm.rvs(0.0004,0.0003))})])

  add_city(Pakistan, 'Skardu',15000,325000,17, [("Skardu",'Ghanche',{'weight':abs(norm.rvs(0.0004,0.0003))}), ("Skardu",'Hunza-Nagar',{'weight':abs(norm.rvs(0.0004,0.0003))}),
                                              ("Skardu",'Gilgit',{'weight':abs(norm.rvs(0.0004,0.0003))}), ("Skardu",'Astore',{'weight':abs(norm.rvs(0.0004,0.0003))}),
                                              ("Skardu",'Kashmir',{'weight':abs(norm.rvs(0.0004,0.0003))})])

  add_city(Pakistan, 'Astore',8657,100000,0, [("Astore",'Skardu',{'weight':abs(norm.rvs(0.0004,0.0003))}), ("Astore",'Gilgit',{'weight':abs(norm.rvs(0.0004,0.0003))}),
                                              ("Astore",'Diamer',{'weight':abs(norm.rvs(0.0004,0.0003))}), ("Astore",'Kashmir',{'weight':abs(norm.rvs(0.0004,0.0003))})])

  add_city(Pakistan, 'Diamer',10936,177000,0, [("Diamer",'Astore',{'weight':abs(norm.rvs(0.0004,0.0003))}), ("Diamer",'Gilgit',{'weight':abs(norm.rvs(0.0004,0.0003))}),
                                              ("Diamer",'Ghizer',{'weight':abs(norm.rvs(0.0004,0.0003))}), ("Diamer",'Kashmir',{'weight':abs(norm.rvs(0.0004,0.0003))}),
                                              ("Diamer",'Lakki Marwat',{'weight':abs(norm.rvs(0.0004,0.0003))}), ("Diamer",'Malakand',{'weight':abs(norm.rvs(0.0004,0.0003))})])

  add_city(Pakistan, 'Ghizer',9635,160000,0,[("Ghizer",'Hunza-Nagar',{'weight':abs(norm.rvs(0.0004,0.0003))}), ("Ghizer",'Gilgit',{'weight':abs(norm.rvs(0.0004,0.0003))}),
                                            ("Ghizer",'Diamer',{'weight':abs(norm.rvs(0.0004,0.0003))}), ("Ghizer",'Chitral',{'weight':abs(norm.rvs(0.0004,0.0003))}),
                                            ("Ghizer",'Lakki Marwat',{'weight':abs(norm.rvs(0.0004,0.0003))}), ("Ghizer",'Swabi',{'weight':abs(norm.rvs(0.0004,0.0003))})])

  add_city(Pakistan, 'Gilgit',1155,230000,49, [("Gilgit",'Skardu',{'weight':abs(norm.rvs(0.0004,0.0003))}),("Gilgit",'Astore',{'weight':abs(norm.rvs(0.0004,0.0003))}),
                                              ("Gilgit",'Hunza-Nagar',{'weight':abs(norm.rvs(0.0004,0.0003))}), ("Gilgit",'Diamer',{'weight':abs(norm.rvs(0.0004,0.0003))}),
                                              ("Gilgit",'Ghizer',{'weight':abs(norm.rvs(0.0004,0.0003))})])

  add_city(Pakistan, 'Hunza-Nagar',25145,180000,5, [("Hunza-Nagar",'Ghizer',{'weight':abs(norm.rvs(0.0004,0.0003))}), ("Hunza-Nagar",'Gilgit',{'weight':abs(norm.rvs(0.0004,0.0003))}),
                                                    ("Hunza-Nagar",'Skardu',{'weight':abs(norm.rvs(0.0004,0.0003))})])
  
  """ KPK """

  add_city(Pakistan, 'Abbottabad',1967,1332912,0, [("Abbottabad",'Haripur',{'weight':abs(norm.rvs(0.0007,0.0003))}), 
                                                  ("Abbottabad",'Malakand',{'weight':abs(norm.rvs(0.0007,0.0003))}), 
                                                  ("Abbottabad",'Rawalpindi',{'weight':abs(norm.rvs(0.0007,0.0003))}), 
                                                  ("Abbottabad",'Kashmir',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Bannu',1227,1167892,0, [("Bannu",'Upper Kohistan',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Bannu",'Karak',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Bannu",'FR Bannu',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Bannu",'FR Lakki Marwat',{'weight':abs(norm.rvs(0.0007,0.0003))})] )

  add_city(Pakistan, 'Battagram',1301,476612,0, [("Battagram",'Lakki Marwat',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Battagram",'Malakand',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Battagram",'Peshawer',{'weight':abs(norm.rvs(0.0007,0.0003))})])

  add_city(Pakistan, 'Buner',1865,897319,2, [("Buner",'Lower Dir',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                            ("Buner",'Mansehra',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                            ("Buner",'Shangla',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                            ("Buner",'Haripur',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                            ("Buner",'Malakand',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                            ("Buner",'Peshawer',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                            ("Buner",'Swabi',{'weight':abs(norm.rvs(0.0007,0.0003))})])

  add_city(Pakistan, 'Charsadda',996,1616198,1, [("Charsadda",'Lower Dir',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Charsadda",'Mansehra',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Charsadda",'Nowshera',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Charsadda",'Mardan',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Charsadda",'Mohmand',{'weight':abs(norm.rvs(0.0007,0.0003))})])


  add_city(Pakistan, 'Chitral',14850,447362,0, [("Chitral",'Tank',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Chitral",'Swabi',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Chitral",'Ghizer',{'weight':abs(norm.rvs(0.0007,0.0003))})])


  add_city(Pakistan, 'Dera Ismail Khan',7326,1627132,53, [("Dera Ismail Khan",'Swat',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                          ("Dera Ismail Khan",'Upper Kohistan',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                          ("Dera Ismail Khan",'Bhakkar',{'weight':abs(norm.rvs(0.0007,0.0003))}), 
                                                          ("Dera Ismail Khan",'Mianwali',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                          ("Dera Ismail Khan",'Layyah',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                          ("Dera Ismail Khan",'Dera Ghazi Khan',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                          ("Dera Ismail Khan",'FR Dera Ismail Khan',{'weight':abs(norm.rvs(0.0007,0.0003))})]) 

  add_city(Pakistan, 'Hangu',1097,518798,1, [("Hangu",'Karak',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                            ("Hangu",'Kohat',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                            ("Hangu",'Orakzai',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                            ("Hangu",'Kurram',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                            ("Hangu",'FR Bannu',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                            ("Hangu",'North Waziristan',{'weight':abs(norm.rvs(0.0007,0.0003))})])

  add_city(Pakistan, 'Haripur',1725,1003031,0, [("Haripur",'Shangla',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Haripur",'Abbottabad',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Haripur",'Malakand',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Haripur",'Buner',{'weight':abs(norm.rvs(0.0007,0.0003))}), 
                                              ("Haripur",'Islamabad',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Haripur",'Attock',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Haripur",'Rawalpindi',{'weight':abs(norm.rvs(0.0007,0.0003))})])

  add_city(Pakistan, 'Karak',3372,706299,1, [("Karak",'Mianwali',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Karak",'Hangu',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Karak",'Bannu',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Karak",'Kohat',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Karak",'Upper Kohistan',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Karak",'FR Bannu',{'weight':abs(norm.rvs(0.0007,0.0003))})])


  add_city(Pakistan, 'Kohat',2545,993874,0, [("Kohat",'Hangu',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                            ("Kohat",'Karak',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                            ("Kohat",'Mianwali',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                            ("Kohat",'Attock',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                            ("Kohat",'FR Peshawer',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                            ("Kohat",'FR Kohat',{'weight':abs(norm.rvs(0.0007,0.0003))})])


  add_city(Pakistan, 'Lakki Marwat',2545,993874,0, [("Lakki Marwat",'Swabi',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                    ("Lakki Marwat",'Peshawer',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                    ("Lakki Marwat",'Battagram',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                    ("Lakki Marwat",'Malakand',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                    ("Lakki Marwat",'Diamer',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                    ("Lakki Marwat",'Ghizer',{'weight':abs(norm.rvs(0.0007,0.0003))})])



                                                    
  add_city(Pakistan, 'Upper Kohistan',7492,306337,0, [("Upper Kohistan",'Swat',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                      ("Upper Kohistan",'Dera Ismail Khan',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                      ("Upper Kohistan",'Bannu',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                      ("Upper Kohistan",'Karak',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                      ("Upper Kohistan",'Mianwali',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                      ("Upper Kohistan",'FR Lakki Marwat',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                      ("Upper Kohistan",'FR Tank',{'weight':abs(norm.rvs(0.0007,0.0003))})])

  #Estimated lower kohistan area using the map since data wasn't available
  add_city(Pakistan, 'Lower Kohistan',3000,202913,0, [("Lower Kohistan",'Tank',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                  ("Lower Kohistan",'Lower Dir',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                  ("Lower Kohistan",'Swabi',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                  ("Lower Kohistan",'Bajaur',{'weight':abs(norm.rvs(0.0007,0.0003))})])



  add_city(Pakistan, 'Lower Dir',1582,1435917,0, [("Lower Dir",'Lower Kohistan',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Lower Dir",'Charsadda',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Lower Dir",'Mansehra',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Lower Dir",'Swabi',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Lower Dir",'Buner',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Lower Dir",'Bajaur',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Lower Dir",'Mohmand',{'weight':abs(norm.rvs(0.0007,0.0003))})])

  add_city(Pakistan, 'Malakand',	952,720295,0, [("Malakand",'Abbottabad',{'weight':abs(norm.rvs(0.0007,0.0003))}), 
                                                ("Malakand",'Battagram',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Malakand",'Lakki Marwat',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Malakand",'Haripur',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Malakand",'Buner',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Malakand",'Kashmir',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Malakand",'Diamer',{'weight':abs(norm.rvs(0.0007,0.0003))})])
                                                    
  add_city(Pakistan, 'Mansehra',4579,1556460,3, [("Mansehra",'Buner',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                ("Mansehra",'Mardan',{'weight':abs(norm.rvs(0.0007,0.0003))}), 
                                                ("Mansehra",'Lower Dir',{'weight':abs(norm.rvs(0.0007,0.0003))}), 
                                                ("Mansehra",'Shangla',{'weight':abs(norm.rvs(0.0007,0.0003))}), 
                                                ("Mansehra",'Charsadda',{'weight':abs(norm.rvs(0.0007,0.0003))})])  

                                                                                                        
  add_city(Pakistan, 'Mardan',1632,2373061,6, [("Mardan",'Shangla',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Mardan",'Mansehra',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Mardan",'Charsadda',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Mardan",'Nowshera',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Mardan",'Kohat',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Mardan",'FR Peshawer',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Mardan",'Attock',{'weight':abs(norm.rvs(0.0007,0.0003))})])


  add_city(Pakistan, 'Nowshera',1748,1518540,0, [("Nowshera",'Charsadda',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Nowshera",'Mardan',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Nowshera",'Khyber',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Nowshera",'Mohmand',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Nowshera",'FR Peshawer',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Nowshera",'FR Tank',{'weight':abs(norm.rvs(0.001,0.0005))})])


  add_city(Pakistan, 'Peshawer',1257,4269079,12, [("Peshawer",'Swabi',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Peshawer",'Lakki Marwat',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Peshawer",'Battagram',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Peshawer",'Malakand',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                                ("Peshawer",'Buner',{'weight':abs(norm.rvs(0.001,0.0005))})])



  add_city(Pakistan, 'Shangla',1586,757810,0, [("Shangla",'Mardan',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                    ("Shangla",'Haripur',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                    ("Shangla",'Buner',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                    ("Shangla",'Mansehra',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                                    ("Shangla",'Attock',{'weight':abs(norm.rvs(0.0007,0.0003))})])


  add_city(Pakistan, 'Swabi',1543,1624616,0, [("Swabi",'Tank',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Swabi",'Lakki Marwat',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Swabi",'Peshawer',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Swabi",'Buner',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Swabi",'Lower Dir',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Swabi",'Lower Kohistan',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Swabi",'Chitral',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                              ("Swabi",'Ghizer',{'weight':abs(norm.rvs(0.0007,0.0003))})])


  add_city(Pakistan, 'Swat',5337,2309570,0, [("Swat",'Upper Kohistan',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                            ("Swat",'Dera Ismail Khan',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                            ("Swat",'FR Tank',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                            ("Swat",'South Waziristan',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                            ("Swat",'Dera Ismail Khan',{'weight':abs(norm.rvs(0.0007,0.0003))})])


  add_city(Pakistan, 'Tank',1679,39188,0, [("Tank",'Chitral',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                          ("Tank",'Swabi',{'weight':abs(norm.rvs(0.0007,0.0003))}),
                                          ("Tank",'Lower Kohistan',{'weight':abs(norm.rvs(0.0007,0.0003))})])

  """ Balochistan """

  add_city(Pakistan, 'Awaran',29510,121680,0, [("Awaran",'Panjgur',{'weight':abs(norm.rvs(0.001,0.0005))}),("Awaran",'Kech',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Awaran",'Gwadar',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Awaran",'Lasbela',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Awaran",'Khuzdar',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Awaran",'Kharan',{'weight':abs(norm.rvs(0.001,0.0005))})] )

  add_city(Pakistan, 'Barkhan',3514,171556,0, [("Barkhan",'Dera Bugti',{'weight':abs(norm.rvs(0.001,0.0005))}),("Barkhan",'Kohlu',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                            ("Barkhan",'Loralai',{'weight':abs(norm.rvs(0.001,0.0005))}),("Barkhan",'Musakhel',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                            ("Barkhan",'Dera Ghazi Khan',{'weight':abs(norm.rvs(0.001,0.0005))})])

  add_city(Pakistan, 'Bolan',7499,288056,0, [("Bolan",'Nasirabad',{'weight':abs(norm.rvs(0.001,0.0005))}),("Bolan",'Jhal Magsi',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Bolan",'Khuzdar',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Bolan",'Kalat',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Bolan",'Mastang',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Bolan",'Harnai',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Bolan",'Sibi',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Bolan",'Kohlu',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Bolan",'Dera Bugti',{'weight':abs(norm.rvs(0.001,0.0005))})])


  add_city(Pakistan, 'Chagai',44748,226008,0, [("Chagai",'Nushki',{'weight':abs(norm.rvs(0.001,0.0005))}),("Chagai",'Kharan',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                            ("Chagai",'Washuk',{'weight':abs(norm.rvs(0.001,0.0005))})])

  add_city(Pakistan, 'Dera Bugti',10160,312603,0, [("Dera Bugti",'Barkhan',{'weight':abs(norm.rvs(0.001,0.0005))}),("Dera Bugti",'Kohlu',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Dera Bugti",'Bolan',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Dera Bugti",'Nasirabad',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Dera Bugti",'Jafarabad',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Dera Bugti",'Dera Ghazi Khan',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Dera Bugti",'Kashmore',{'weight':abs(norm.rvs(0.001,0.0005))})])

  add_city(Pakistan, 'Gwadar',12637,263514,0, [("Gwadar",'Kech',{'weight':abs(norm.rvs(0.001,0.0005))}),("Gwadar",'Awaran',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                            ("Gwadar",'Lasbela',{'weight':abs(norm.rvs(0.001,0.0005))})])

  add_city(Pakistan, 'Harnai',4096,97017 ,0, [("Harnai",'Sibi',{'weight':abs(norm.rvs(0.001,0.0005))}),("Harnai",'Bolan',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Harnai",'Mastang',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Harnai",'Quetta',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Harnai",'Ziarat',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Harnai",'Loralai',{'weight':abs(norm.rvs(0.001,0.0005))})] )


  add_city(Pakistan, 'Jafarabad',2445,513813,0,[("Jafarabad",'Nasirabad',{'weight':abs(norm.rvs(0.001,0.0005))}),("Jafarabad",'Jhal Magsi',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                            ("Jafarabad",'Dera Bugti',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Jafarabad",'Jacobabad',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                            ("Jafarabad",'Qambar Shahdadkot',{'weight':abs(norm.rvs(0.001,0.0005))})])
          
  add_city(Pakistan, 'Jhal Magsi',3615,149225,0, [("Jhal Magsi",'Khuzdar',{'weight':abs(norm.rvs(0.001,0.0005))}),("Jhal Magsi",'Bolan',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Jhal Magsi",'Nasirabad',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Jhal Magsi",'Jafarabad',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Jhal Magsi",'Qambar Shahdadkot',{'weight':abs(norm.rvs(0.001,0.0005))})] )

  add_city(Pakistan, 'Kalat',6622,412232,0, [("Kalat",'Mastang',{'weight':abs(norm.rvs(0.001,0.0005))}),("Kalat",'Nushki',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Kalat",'Kharan',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Kalat",'Khuzdar',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Kalat",'Bolan',{'weight':abs(norm.rvs(0.001,0.0005))})])


  add_city(Pakistan, 'Kech',22539,909116 ,0, [("Kech",'Gwadar',{'weight':abs(norm.rvs(0.001,0.0005))}),("Kech",'Awaran',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Kech",'Panjgur',{'weight':abs(norm.rvs(0.001,0.0005))})])


  add_city(Pakistan, 'Kharan',8958,156152,0, [("Kharan",'Chagai',{'weight':abs(norm.rvs(0.001,0.0005))}),("Kharan",'Washuk',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Kharan",'Panjgur',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Kharan",'Awaran',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Kharan",'Khuzdar',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Kharan",'Kalat',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Kharan",'Nushki',{'weight':abs(norm.rvs(0.001,0.0005))})] )
          

  add_city(Pakistan, 'Kohlu',7610,214350,0, [("Kohlu",'Barkhan',{'weight':abs(norm.rvs(0.001,0.0005))}),("Kohlu",'Dera Bugti',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Kohlu",'Bolan',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Kohlu",'Sibi',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Kohlu",'Loralai',{'weight':abs(norm.rvs(0.001,0.0005))})])
  
  add_city(Pakistan, 'Khuzdar',35380,802207,0, [("Khuzdar",'Jhal Magsi',{'weight':abs(norm.rvs(0.001,0.0005))}),("Khuzdar",'Kalat',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Khuzdar",'Kharan',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Khuzdar",'Awaran',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Khuzdar",'Lasbela',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Khuzdar",'Dadu',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Khuzdar",'Qambar Shahdadkot',{'weight':abs(norm.rvs(0.001,0.0005))})])

  add_city(Pakistan, 'Killa Abdullah',3293,757578,0, [("Killa Abdullah",'Pishin',{'weight':abs(norm.rvs(0.001,0.0005))}),("Killa Abdullah",'Quetta',{'weight':abs(norm.rvs(0.001,0.0005))})] )

  add_city(Pakistan, 'Killa Saifullah',6831,342814,0, [("Killa Saifullah",'Zhob',{'weight':abs(norm.rvs(0.001,0.0005))}),("Killa Saifullah",'Loralai',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Killa Saifullah",'Ziarat',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Killa Saifullah",'Pishin',{'weight':abs(norm.rvs(0.001,0.0005))})])
        
  add_city(Pakistan, 'Lasbela',15153,574292,0, [("Lasbela",'Awaran',{'weight':abs(norm.rvs(0.001,0.0005))}),("Lasbela",'Gwadar',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Lasbela",'Khuzdar',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Lasbela",'Dadu',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Lasbela",'Jamshoro',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Lasbela",'Karachi',{'weight':abs(norm.rvs(0.001,0.0005))})])


  add_city(Pakistan, 'Loralai',9830,397400 ,0, [("Loralai",'Barkhan',{'weight':abs(norm.rvs(0.001,0.0005))}),("Loralai",'Kohlu',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Loralai",'Sibi',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Loralai",'Harnai',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Loralai",'Ziarat',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Loralai",'Killa Saifullah',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Loralai",'Zhob',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Loralai",'Musakhel',{'weight':abs(norm.rvs(0.001,0.0005))})])
          
  add_city(Pakistan, 'Mastang',5896,133983,0, [("Mastang",'Kalat',{'weight':abs(norm.rvs(0.001,0.0005))}),("Mastang",'Nushki',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Mastang",'Quetta',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Mastang",'Harnai',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Mastang",'Bolan',{'weight':abs(norm.rvs(0.001,0.0005))})])
          
  add_city(Pakistan, 'Musakhel',5728,167017,0, [("Musakhel",'Barkhan',{'weight':abs(norm.rvs(0.001,0.0005))}),("Musakhel",'Loralai',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Musakhel",'Zhob',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Musakhel",'FR Dera Ismail Khan',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Musakhel",'Dera Ghazi Khan',{'weight':abs(norm.rvs(0.001,0.0005))})])
          
          
  add_city(Pakistan, 'Nasirabad',3387,490538,0, [("Nasirabad",'Dera Bugti',{'weight':abs(norm.rvs(0.001,0.0005))}),("Nasirabad",'Jafarabad',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Nasirabad",'Jhal Magsi',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Nasirabad",'Bolan',{'weight':abs(norm.rvs(0.001,0.0005))})])
          
  add_city(Pakistan, 'Nushki',5797,178796,0, [("Nushki",'Chagai',{'weight':abs(norm.rvs(0.001,0.0005))}),("Nushki",'Kharan',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Nushki",'Mastang',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Nushki",'Quetta',{'weight':abs(norm.rvs(0.001,0.0005))})])


  add_city(Pakistan, 'Panjgur',16891,316385,0, [("Panjgur",'Awaran',{'weight':abs(norm.rvs(0.001,0.0005))}),("Panjgur",'Kech',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Panjgur",'Kharan',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Panjgur",'Washuk',{'weight':abs(norm.rvs(0.001,0.0005))})])
          
          
  add_city(Pakistan, 'Pishin',7819,736481,0, [("Pishin",'Killa Saifullah',{'weight':abs(norm.rvs(0.001,0.0005))}),("Pishin",'Killa Abdullah',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Pishin",'Ziarat',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Pishin",'Quetta',{'weight':abs(norm.rvs(0.001,0.0005))})])
          
  add_city(Pakistan, 'Quetta',2653,2275699,101, [("Quetta",'Killa Abdullah',{'weight':abs(norm.rvs(0.001,0.0005))}),("Quetta",'Pishin',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Quetta",'Ziarat',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Quetta",'Harnai',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Quetta",'Mastang',{'weight':abs(norm.rvs(0.001,0.0005))})])

      
  add_city(Pakistan, 'Sherani',2800,153116,0, [("Sherani",'Zhob',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Sherani",'South Waziristan',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Sherani",'FR Dera Ismail Khan',{'weight':abs(norm.rvs(0.001,0.0005))})])
          
          
          
  add_city(Pakistan, 'Sibi',7796,135572,0, [("Sibi",'Harnai',{'weight':abs(norm.rvs(0.001,0.0005))}),("Sibi",'Bolan',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Sibi",'Kohlu',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Sibi",'Loralai',{'weight':abs(norm.rvs(0.001,0.0005))})])


  add_city(Pakistan, 'Washuk',29510,176206 ,0, [("Washuk",'Sherani',{'weight':abs(norm.rvs(0.001,0.0005))}),("Washuk",'Musakhel',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Washuk",'Panjgur',{'weight':abs(norm.rvs(0.001,0.0005))})])



          
          
  add_city(Pakistan, 'Zhob',20297,310544,0, [("Zhob",'Killa Saifullah',{'weight':abs(norm.rvs(0.001,0.0005))}),("Zhob",'Musakhel',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Zhob",'Loralai',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Zhob",'Pishin',{'weight':abs(norm.rvs(0.001,0.0005))})])
        
          
          
  add_city(Pakistan, 'Ziarat',1489,160422,0, [("Ziarat",'Pishin',{'weight':abs(norm.rvs(0.001,0.0005))}),("Ziarat",'Quetta',{'weight':abs(norm.rvs(0.001,0.0005))}),
                                              ("Ziarat",'Loralai',{'weight':abs(norm.rvs(0.001,0.0005))}), ("Ziarat",'Harnai',{'weight':abs(norm.rvs(0.001,0.0005))})])
  
  return Pakistan







In [68]:
#Run the simulation for an X number of days. X is determined by how many times
#you run the loop
for i in range(1):
  Pakistan = make_Pakistan()
  simulate_day(Pakistan,0.5,0.3)

#city_est = [[] for x in range(23)]
  
total = 0
for i in Pakistan.nodes:
  total += Pakistan.nodes[i]['cases']
  print('Number of cases in ', i, ' are ', Pakistan.nodes[i]['cases'])

print('Total cases in Pakistan are', total)

Number of cases in  Badin  are  0
Number of cases in  Thatta  are  6
Number of cases in  Tando Muhammad Khan  are  0
Number of cases in  Hyderabad  are  2
Number of cases in  Tando Allahyar  are  0
Number of cases in  Mirpur Khas  are  0
Number of cases in  Tharparker  are  0
Number of cases in  Dadu  are  2
Number of cases in  Qambar Shahdadkot  are  0
Number of cases in  Larkana  are  1
Number of cases in  Naushahro Firoz  are  0
Number of cases in  Jamshoro  are  1
Number of cases in  Khuzdar  are  0
Number of cases in  Lasbela  are  5
Number of cases in  Ghotki  are  16
Number of cases in  Kashmore  are  4
Number of cases in  Sukkur  are  445
Number of cases in  Rahim Yar Khan  are  1
Number of cases in  Matiari  are  0
Number of cases in  Sanghar  are  0
Number of cases in  Jacobabad  are  0
Number of cases in  Shikarpur  are  12
Number of cases in  Jafarabad  are  0
Number of cases in  Nawabshah  are  0
Number of cases in  Karachi  are  541
Number of cases in  Dera Ghazi Khan  ar